## Training Bayesian Network

### Get updated precision matrix from data

In [1]:
# Import libraries 
import pandas as pd 
import numpy as np

In [2]:
# Load CO2 and Ethylene gas mixture file
header_names = ["Time", "CO2", "Ethylene", "Sensor1", "Sensor2", "Sensor3", "Sensor4", "Sensor5", "Sensor6", "Sensor7", "Sensor8", "Sensor9", "Sensor10", "Sensor11", "Sensor12", "Sensor13", "Sensor14", "Sensor15", "Sensor16"]
df = pd.read_csv("data/gas-mixture/ethylene_CO.txt", delim_whitespace=True, skiprows=[0], header=None, names=header_names)

# Set time column as index
df = df.set_index("Time")

In [3]:
# Check data frame
df.head()

,CO2,Ethylene,Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6,Sensor7,Sensor8,Sensor9,Sensor10,Sensor11,Sensor12,Sensor13,Sensor14,Sensor15,Sensor16
Time,,,,,,,,,,,,,,,,,,
0.00,0.0,0.0,-50.85,-1.95,-41.82,1.30,-4.07,-28.73,-13.49,-3.25,55139.95,50669.50,9626.26,9762.62,24544.02,21420.68,7650.61,6928.42
0.01,0.0,0.0,-49.40,-5.53,-42.78,0.49,3.58,-34.55,-9.59,5.37,54395.77,50046.91,9433.20,9591.21,24137.13,20930.33,7498.79,6800.66
0.01,0.0,0.0,-40.04,-16.09,-27.59,0.00,-7.16,-42.14,-12.52,-5.86,53960.02,49299.30,9324.40,9449.81,23628.90,20504.94,7369.67,6697.47
0.03,0.0,0.0,-47.14,-10.57,-32.28,4.40,-11.22,-37.94,-7.16,-1.14,53047.71,48907.00,9170.64,9305.58,23101.66,20101.42,7285.13,6578.52
0.04,0.0,0.0,-33.58,-20.79,-33.25,6.03,3.42,-34.22,-14.46,8.31,52700.28,48330.96,9073.64,9163.47,22689.54,19694.07,7156.74,6468.32


In [4]:
df = df.drop(['Sensor2','Sensor4','Sensor6','Sensor8','Sensor9','Sensor10','Sensor11','Sensor12','Sensor13','Sensor14','Sensor15','Sensor16'], axis=1)

In [5]:
# Transform data frame in numpy matrix
# data = (df.head(1000)).values
data = df.values

In [6]:
# Calculate mean of each column
x_mean = data.mean(0)

In [7]:
# Calculate s
s = data-x_mean
s = (s.T).dot(s)

In [8]:
# Calculate M
M = data.size

In [9]:
# Assuming prior ignorance. v = 0 and beta = 0
beta_update = s
mi_update = x_mean
v_update = M
alpha_update = M-1

In [10]:
# number of random variables
n = 6 

# Calculate Phi_Update
phi_update = (v_update+1)/(v_update*(alpha_update-n+1))
phi_update *= beta_update

In [11]:
# Calculate Precision Update
precision_update = np.linalg.inv(phi_update)

### Calculate Symbolic Precision Matrix

In [12]:
import sympy as sp

In [13]:
# Startup and configuration
n = 6          # 6 random variables
b = [None]*n   # list to initialize bij symbols
v = [None]*n   # list to initialize vi  symbols (variance)

In [14]:
# Create all bij symbols (from b11 to b1818)
for i in range(n):
    symbol_string = ''
    for j in range(i+1):
        symbol_string += 'b' + str(i+1) + str(j+1) + ' '
    b[i] = sp.symbols(symbol_string)

In [15]:
# Start B matrix using bij symbols
B = sp.zeros(n,n)
B[0, 0] = b[0]       # first symbols is exception not being a list of list
for i in range(1,n):
    for j in range(i+1):
        B[i,j] = b[i][j]

In [16]:
# Create all v, variance symbols (from v1 to v18)
for i in range(n):
    v[i] = sp.symbols(r'\sigma' + str(i+1))

In [17]:
T = sp.zeros(1,1)
T[0,0] = (1/v[0])
for i in range (1, n):
    # Initialize ti
#    t = (1/v[i])**2
    t = (1/v[i]) # For simplicity variance will be represented without the square
    tt = sp.zeros(1,1)
    tt[0,0] = t
    
    # Initialize  
    TMP = sp.zeros(i+1,i+1)
    
    ## Populate                                                ## row, col
    TMP[0:i,   0:i]   = T + t*sp.Transpose(B[i,0:i])*B[i,0:i]  ## [0 , 0]
    TMP[0:i,   i:i+1] = -sp.Transpose(t*B[i,0:i])              ## [0 , 1]
    TMP[i:i+1, 0:i]   = -t*B[i,0:i]                            ## [1 , 0]
    TMP[i:i+1, i:i+1] = tt                                     ## [1 , 1]
    
    
    T = TMP
T

Matrix([
[b61**2/\sigma6 + b51**2/\sigma5 + b41**2/\sigma4 + b31**2/\sigma3 + b21**2/\sigma2 + 1/\sigma1, b61*b62/\sigma6 + b51*b52/\sigma5 + b41*b42/\sigma4 + b31*b32/\sigma3 - b21/\sigma2, b61*b63/\sigma6 + b51*b53/\sigma5 + b41*b43/\sigma4 - b31/\sigma3, b61*b64/\sigma6 + b51*b54/\sigma5 - b41/\sigma4, b61*b65/\sigma6 - b51/\sigma5, -b61/\sigma6],
[           b61*b62/\sigma6 + b51*b52/\sigma5 + b41*b42/\sigma4 + b31*b32/\sigma3 - b21/\sigma2,       b62**2/\sigma6 + b52**2/\sigma5 + b42**2/\sigma4 + b32**2/\sigma3 + 1/\sigma2, b62*b63/\sigma6 + b52*b53/\sigma5 + b42*b43/\sigma4 - b32/\sigma3, b62*b64/\sigma6 + b52*b54/\sigma5 - b42/\sigma4, b62*b65/\sigma6 - b52/\sigma5, -b62/\sigma6],
[                             b61*b63/\sigma6 + b51*b53/\sigma5 + b41*b43/\sigma4 - b31/\sigma3,                   b62*b63/\sigma6 + b52*b53/\sigma5 + b42*b43/\sigma4 - b32/\sigma3,      b63**2/\sigma6 + b53**2/\sigma5 + b43**2/\sigma4 + 1/\sigma3, b63*b64/\sigma6 + b53*b54/\sigma5 - b43/\sigma4, b63*b

### Solve Precision Matrix (get parameters $\sigma_{ij}^*$ and $b_{ij}$)

In [18]:
# Start Equation list
counter = 20
EQ = sp.zeros(21, 1)
for i in range(0,n):
    for j in range(i,n):
        # EQ[(i-1)*n + j] = T[i][j]-precision_update[i][j]        
        EQ[counter] = T[i, j]-precision_update[i, j]
        counter -= 1

In [19]:
EQ

Matrix([
[                                                                                      -0.00025352450732528 + 1/\sigma6],
[                                                                                    0.000177985823686287 - b65/\sigma6],
[                                                                     -0.00045326362154776 + b65**2/\sigma6 + 1/\sigma5],
[                                                                                     0.00026064186856562 - b64/\sigma6],
[                                                                 -0.000127088112756583 + b64*b65/\sigma6 - b54/\sigma5],
[                                                   -0.000282334582924682 + b64**2/\sigma6 + b54**2/\sigma5 + 1/\sigma4],
[                                                                                    -3.78203310293082e-5 - b63/\sigma6],
[                                                                   9.36628917042551e-5 + b63*b65/\sigma6 - b53/\sigma5],
[              

In [20]:
sp.solve(EQ, dict=True, manual=True)

[{\sigma1: 4308.22078756868,
  \sigma2: 5.25982684986330,
  \sigma3: 59751.6290327189,
  \sigma4: 205779.945162881,
  \sigma5: 3045.90715851811,
  \sigma6: 3944.39184814969,
  b21: -0.00166720448648686,
  b31: 3.17862904387405,
  b32: 17.4238281420638,
  b41: 0.177011154774773,
  b42: 167.392468979708,
  b43: 0.740523255388914,
  b51: 0.421009294646619,
  b52: 4.14896289255278,
  b53: 0.204414746258858,
  b54: 0.170249228386767,
  b61: 0.182099408047204,
  b62: 24.0096412107577,
  b63: -0.149178205406326,
  b64: 1.02807366165673,
  b65: 0.702045832034398}]